# Qwen2.5-7B-Instruct to predict IPV and utilize DetectIPV Paper Evaluation Pipeline
Key tasks:
* Binary IPV detection (IPV vs NOT_IPV)
* Multi-label subtype detection (Physical, Emotional, Sexual)

**Key Evaluation Components from the Paper**
| Figure/Table | What it Shows |
|---------------|----------------|
| **Fig. 2(a)** | ROC curve for *General Violence Model* |
| **Fig. 2(b–c)** | “Waterfall” plot: sentences sorted by predicted confidence; color = ground-truth subtype |
| **Fig. 3(a)** | AUROC of Type-Specific vs General Models |
| **Fig. 3(b–e)** | More waterfall plots for type-specific models |
| **Fig. 4** | Radial visualization of confidence in 3-D type space |
| **Table 4** | AUROC for models trained on different negative examples |

## 1. Prompt Design

To evaluate how prompt structure influences LLM performance on IPV detection, I used **five prompting strategies** for both binary (IPV vs. NOT_IPV) and multi-label (Physical, Emotional, Sexual) tasks.  
This allows testing how reasoning depth, context, and self-reflection affect accuracy, AUC, and interpretability.

| Prompt Type | Description | Advantage |
|--------------|-------------|------------|
| **Zero-Shot** | Direct question without examples or reasoning. | Tests model’s innate understanding of IPV cues. |
| **Few-Shot** | Includes short examples before prediction. | Leverages in-context learning and improves clarity. |
| **Chain-of-Thought** | Prompts reasoning steps before final label. | Makes predictions more explainable and robust. |
| **Meta-Prompt** | Frames model as an IPV analyst with reflective reasoning. | Encourages careful, context-aware decisions. |
| **Self-Consistency** | Runs multiple internal judgments and votes. | Improves stability and reduces random errors. |

These prompt variants help identify which instruction style yields the **most reliable and interpretable results** for both general and type-specific IPV detection.

### 1.1 Binary IPV Detection Prompts

In [ ]:
# (1) Zero-Shot — minimal output
prompt_binary_zeroshot = """
You are identifying whether a sentence describes Intimate Partner Violence (IPV).

Label exactly one of: IPV, NOT_IPV.

Sentence: "{text}"

Output JSON only:
{"label": "<IPV or NOT_IPV>"}
"""

# (2) Few-Shot — minimal output
prompt_binary_fewshot = """
Classify whether this sentence shows Intimate Partner Violence (IPV).

Examples:
1. "He threw a plate at me when we argued." → IPV
2. "She criticizes me constantly and isolates me from friends." → IPV
3. "We had a disagreement about chores but talked it through calmly." → NOT_IPV

Now analyze:
"{text}"

Output JSON only:
{"label": "<IPV or NOT_IPV>"}
"""

# (3) Chain-of-Thought — internal only, minimal output
prompt_binary_cot = """
Decide if the sentence describes Intimate Partner Violence (IPV).
Think through the decision step by step **internally** and do not reveal your reasoning.

Sentence: "{text}"

Output JSON only (no explanations):
{"label": "<IPV or NOT_IPV>"}
"""

# (4) Meta-Prompt — internal reflection, minimal output
prompt_binary_meta = """
You are a cautious social-behavioral analyst detecting signs of Intimate Partner Violence (IPV).
Reflect carefully **internally**; do not include your reasoning in the answer.

Sentence: "{text}"

Output JSON only:
{"label": "<IPV or NOT_IPV>"}
"""

# (5) Self-Consistency — internal voting, minimal output
prompt_binary_selfconsistency = """
Evaluate the sentence for IPV three times **internally** and choose the majority label.
Do not reveal intermediate thoughts or votes.

Sentence: "{text}"

Output JSON only:
{"label": "<IPV or NOT_IPV>"}
"""

### 1.2 Multi-label IPV Type Detection Prompts

In [ ]:
# (1) Zero-Shot — minimal output
prompt_multilabel_zeroshot = """
Identify which forms of Intimate Partner Violence (IPV) appear in the sentence.
Possible labels: Physical, Emotional, Sexual. If none apply, return [].

Sentence: "{text}"

Output JSON only:
{"labels": ["Physical", "Emotional", "Sexual"]}
"""

# (2) Few-Shot — minimal output
prompt_multilabel_fewshot = """
Label the types of Intimate Partner Violence (IPV) in a sentence.
Examples:
1. "He hit me and threw me against the wall." → ["Physical"]
2. "She insults me and controls who I talk to." → ["Emotional"]
3. "He forces me to have sex when I refuse." → ["Sexual"]
4. "We disagree but support each other." → []

Now classify:
"{text}"

Output JSON only:
{"labels": [...]}
"""

# (3) Chain-of-Thought — internal only, minimal output
prompt_multilabel_cot = """
Determine which IPV types are present (Physical, Emotional, Sexual).
Think step by step **internally** and do not reveal your reasoning.

Sentence: "{text}"

Output JSON only (no explanations):
{"labels": ["Physical", "Emotional", "Sexual"]}
"""

# (4) Meta-Prompt — internal reflection, minimal output
prompt_multilabel_meta = """
You are an IPV classification expert. Reflect carefully **internally**; do not include your reasoning.
Possible labels: Physical, Emotional, Sexual. If none apply, return [].

Sentence: "{text}"

Output JSON only:
{"labels": ["Physical", "Emotional", "Sexual"]}
"""

# (5) Self-Consistency — internal voting, minimal output
prompt_multilabel_selfconsistency = """
Evaluate the sentence three times **internally** and choose the final consistent set of labels.
Possible labels: Physical, Emotional, Sexual. If none apply, return [].

Sentence: "{text}"

Output JSON only:
{"labels": ["Physical", "Emotional", "Sexual"]}
"""

## 2. System & Model Setup

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import json
import os
import time

In [ ]:
#FILENAMES
model_name = "Qwen/Qwen2.5-7B-Instruct"
filename = "../Dataset/617points.csv"


#Load Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.


## 3. Generate Predictions

In [ ]:
# ============================================
# Helper: Run model and return decoded text
# ============================================
def run_llm(prompt_text):
    """Feed a prompt into the LLM and return decoded text output."""
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.0,     # deterministic for evaluation
        do_sample=False
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text.strip()

# ============================================
# Flexible run function
# ============================================
def run_prompt(df, task_type="binary", prompt_type="zeroshot", n_samples=10):
    """
    Run one selected prompt (binary or multilabel) on a specified number of samples.
    Saves minimal JSONL: {id, prompt_type, output}.
    """

    # Select correct prompt
    if task_type == "binary":
        prompt_template = binary_prompts[prompt_type]
    elif task_type == "multilabel":
        prompt_template = multilabel_prompts[prompt_type]
    else:
        raise ValueError("task_type must be 'binary' or 'multilabel'.")

    # Subset dataset
    df_subset = df.head(n_samples) if n_samples else df
    print(f"\nRunning {task_type.upper()} → {prompt_type} on {len(df_subset)} samples")

    # Prepare output directory and file
    Path("results").mkdir(exist_ok=True)
    output_file = Path(f"results/{task_type}_{prompt_type}_sample{len(df_subset)}.jsonl")

    # Main loop
    with open(output_file, "w", encoding="utf-8") as f_out:
        for i, row in tqdm(df_subset.iterrows(), total=len(df_subset)):
            text = row["items"] if "items" in df.columns else str(row[0])

            # Format prompt
            prompt_text = f"{prompt_template}".replace("{text}", text)

            # Get model output
            try:
                result_text = run_llm(prompt_text)
            except Exception as e:
                result_text = f"ERROR: {e}"

            # Try to extract JSON cleanly
            try:
                start, end = result_text.find("{"), result_text.rfind("}") + 1
                parsed = json.loads(result_text[start:end])
                output_value = parsed.get("label") or parsed.get("labels") or result_text
            except Exception:
                output_value = result_text

            # Minimal record
            record = {
                "id": int(i),
                "prompt_type": prompt_type,
                "output": output_value
            }

            f_out.write(json.dumps(record) + "\n")

    print(f"Finished {task_type} ({prompt_type}) → Saved to {output_file}")

In [ ]:

# ============================================
# Example usage
# ============================================
df = pd.read_csv(filename)

# Example 1: run few-shot binary prompt on 10 samples
run_prompt(df, task_type="binary", prompt_type="fewshot", n_samples=10)

# Example 2: run meta multi-label prompt on first 20 samples
# run_prompt(df, task_type="multilabel", prompt_type="meta", n_samples=20)

In [ ]:
# ---- Helper: Run model and return raw text ----
def run_llm(prompt_text):
    """Feed a prompt into the LLM and return decoded text output."""
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.0,   # deterministic for evaluation
        do_sample=False
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text.strip()


# Define prompts and metadata
binary_prompts = {"zeroshot": prompt_binary_zeroshot,"fewshot": prompt_binary_fewshot,
    "cot": prompt_binary_cot,"meta": prompt_binary_meta, "selfconsistency": prompt_binary_selfconsistency
}

multilabel_prompts = {
    "zeroshot": prompt_multilabel_zeroshot, "fewshot": prompt_multilabel_fewshot,
    "cot": prompt_multilabel_cot, "meta": prompt_multilabel_meta, "selfconsistency": prompt_multilabel_selfconsistency
}

# Define dataframe
df = pd.read_csv(filename)

# Main generation loop
for task_type, prompts in [("binary", binary_prompts), ("multilabel", multilabel_prompts)]:
    for prompt_type, template in prompts.items():
        output_file = Path(f"results/{task_type}_{prompt_type}.jsonl")
        print(f"\nRunning {task_type.upper()} → {prompt_type} | Saving to {output_file}")

        with open(output_file, "w", encoding="utf-8") as f_out:
            for i, row in tqdm(df.iterrows(), total=len(df)):
                text = row["items"] if "items" in df.columns else row[0]

                # Format the prompt
                prompt_text = f"{template}"
                prompt_text = prompt_text.replace("{text}", text)

                # Get model output
                try:
                    result_text = run_llm(prompt_text)
                except Exception as e:
                    result_text = f"ERROR: {e}"

                # Store record
                record = {
                    "id": int(i),
                    "text": text,
                    "prompt_type": prompt_type,
                    "task_type": task_type,
                    "raw_output": result_text,
                    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
                }

                # Write to JSONL (one per line)
                f_out.write(json.dumps(record) + "\n")

        print(f"Finished {prompt_type} ({task_type}) — saved to {output_file}")



Running BINARY → zeroshot | Saving to results/binary_zeroshot.jsonl


  0%|          | 0/618 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/transformers/pytorch_utils.py:339: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)
  0%|          | 2/618 [05:48<29:50:09, 174.37s/it]


KeyboardInterrupt: 

## 4. Evaluation Pipeline

1. **Prompt Comparison Phase**
   * Design multiple prompt variants for each task:
     - **Binary Prompts:** Different phrasings for IPV vs NOT_IPV classification.  
     - **Multi-label Prompts:** Different phrasings for type-specific labeling (Physical, Emotional, Sexual, None).
   * Run all prompts on the same dataset.
   * Compute for each prompt:
     - Accuracy, F1-score, ROC-AUC (per task or per subtype)
     - Average confidence calibration (mean predicted probability for positives/negatives)
   * Visualize:
     - **Prompt Comparison Table or Bar Plot** showing AUC/F1 across prompts
     - Identify the **best-performing prompt** for each task (Binary, Multi-label)

2. **Final Evaluation with Best Prompt**
   * **Binary (General IPV Model):**
     - Use the best binary prompt to compute final **ROC curve**, **AUC**, **Precision-Recall**, **Accuracy**, and **F1-score**.
     - Plot **ROC Curve** (replicating Fig. 2a).
     - Generate **Waterfall Plot** — sentences sorted by prediction confidence, color-coded by true label (Physical, Emotional, Sexual, Negative).

   * **Multi-label (Type-Specific Models):**
     - Use the best multi-label prompt to compute **per-type ROC/AUC** for Physical, Emotional, and Sexual abuse.
     - Compare type-specific vs. general model performance (bar chart similar to Fig. 3a).
     - Produce **Waterfall Plots** for each subtype showing confidence distribution and overlap between true labels and predictions.

3. **(Optional) Extended Visualization**
   * Create a **3D or Radial Plot** showing confidence magnitudes of all three type-specific predictions for interpretability.